# Verily Workbench Setup

Run this notebook once at the start of each JupyterLab session to initialize workspace environment variables.

**This notebook will:**
1. Create workspace GCS buckets if they don't exist (first-time setup only)
2. Set environment variables for the current session
3. Save config to `/home/jupyter/workspace/.aou_config.json` for use in other notebooks

**Files to upload to Verily Workbench:**
- `00_setup_workspace.ipynb` (this notebook)
- `aou_helpers.py` (helper module - upload to same directory as your notebooks)

**Environment variables set:**
- `GOOGLE_CLOUD_PROJECT` - Your Google Cloud project ID
- `WORKSPACE_CDR` - BigQuery dataset with OMOP CDR data
- `WORKSPACE_BUCKET` - Persistent workspace GCS bucket
- `WORKSPACE_TEMP_BUCKET` - Temporary workspace GCS bucket (auto-deleted after 14 days)

**Time to run:** ~2-5 seconds (longer on first run if creating buckets)

In [ ]:
import os
import json
import subprocess
from typing import Dict

In [ ]:
def ensure_workspace_buckets(verbose: bool = True) -> None:
    """
    Create workspace GCS buckets if they don't exist.

    Creates two buckets:
    - workspace-bucket: Persistent storage for analysis outputs
    - temporary-workspace-bucket: Auto-deleted after 14 days

    Args:
        verbose: If True, print creation status
    """
    buckets_config = [
        {
            'name': 'workspace-bucket',
            'description': 'Primary workspace bucket for storing files',
            'auto_delete_days': None
        },
        {
            'name': 'temporary-workspace-bucket',
            'description': 'Bucket for temporary storage. Auto-cleanup after 14 days.',
            'auto_delete_days': 14
        }
    ]

    for bucket in buckets_config:
        # Try to resolve existing bucket
        resolve_result = subprocess.run(
            ['wb', 'resource', 'resolve', '--name', bucket['name']],
            capture_output=True, text=True
        )

        if resolve_result.returncode != 0:
            # Bucket doesn't exist, create it
            create_cmd = [
                'wb', 'resource', 'create', 'gcs-bucket',
                '--name', bucket['name'],
                '--cloning', 'COPY_NOTHING',
                '--description', bucket['description']
            ]

            if bucket['auto_delete_days']:
                create_cmd.extend(['--auto-delete', str(bucket['auto_delete_days'])])

            subprocess.run(create_cmd, check=True, capture_output=True, text=True)

            if verbose:
                print(f"Created bucket: {bucket['name']}")
        else:
            if verbose:
                print(f"Found existing bucket: {bucket['name']}")

In [ ]:
def setup_aou_env(verbose: bool = True) -> Dict[str, str]:
    """
    Set All of Us workspace environment variables using wb CLI.

    Creates workspace GCS buckets if they don't exist, then sets environment
    variables by extracting workspace configuration dynamically.

    Args:
        verbose: If True, print variables as they're set

    Returns:
        dict: Environment variables that were set

    Raises:
        subprocess.CalledProcessError: If wb CLI commands fail
    """
    # Ensure workspace buckets exist (creates them if needed)
    ensure_workspace_buckets(verbose=verbose)

    # Extract workspace info
    workspace = json.loads(
        subprocess.run(
            ["wb", "workspace", "describe", "--format=json"],
            capture_output=True, text=True, check=True
        ).stdout
    )

    # Extract resources
    resources = json.loads(
        subprocess.run(
            ["wb", "resource", "list", "--format=json"],
            capture_output=True, text=True, check=True
        ).stdout
    )

    # Set Google Cloud project
    os.environ["GOOGLE_CLOUD_PROJECT"] = workspace["googleProjectId"]

    # Initialize CDR (will be set below)
    os.environ["WORKSPACE_CDR"] = ""

    # Set buckets and CDR from resources list
    for r in resources:
        if r["resourceType"] == "GCS_BUCKET":
            # Check temporary bucket first to avoid substring conflicts
            if "temporary-workspace-bucket" in r["id"]:
                os.environ["WORKSPACE_TEMP_BUCKET"] = f"gs://{r['bucketName']}"
            elif "workspace-bucket" in r["id"]:
                os.environ["WORKSPACE_BUCKET"] = f"gs://{r['bucketName']}"

        elif r["resourceType"] in ["BQ_DATASET", "BIGQUERY_DATASET"]:
            # Only set CDR if not already set (use first found)
            if os.environ.get("WORKSPACE_CDR") == "":
                os.environ["WORKSPACE_CDR"] = f"{r['projectId']}.{r['datasetId']}"

    # Collect variables for return
    env_vars = {
        "GOOGLE_CLOUD_PROJECT": os.environ.get("GOOGLE_CLOUD_PROJECT"),
        "WORKSPACE_BUCKET": os.environ.get("WORKSPACE_BUCKET"),
        "WORKSPACE_TEMP_BUCKET": os.environ.get("WORKSPACE_TEMP_BUCKET"),
        "WORKSPACE_CDR": os.environ.get("WORKSPACE_CDR")
    }

    if verbose:
        print("Workspace environment variables set:")
        for key, val in env_vars.items():
            print(f"  {key} = {val}")

    return env_vars

In [ ]:
# Execute setup with verbose output
env = setup_aou_env(verbose=True)

In [ ]:
import json
# Store config file
with open('/home/jupyter/workspace/.aou_config.json', 'w') as f:
    json.dump(env, f)

In [ ]:
with open('/home/jupyter/workspace/.aou_config.json') as f:
    env = json.load(f)

In [ ]:
# Assign to Python variables for easy access
WORKSPACE_CDR = env['WORKSPACE_CDR']
WORKSPACE_BUCKET = env['WORKSPACE_BUCKET']
WORKSPACE_TEMP_BUCKET = env['WORKSPACE_TEMP_BUCKET']
GOOGLE_CLOUD_PROJECT = env['GOOGLE_CLOUD_PROJECT']

print("\nVariables ready for analysis:")
print(f"  CDR: {WORKSPACE_CDR}")
print(f"  Bucket: {WORKSPACE_BUCKET}")
print(f"  Temp Bucket: {WORKSPACE_TEMP_BUCKET}")
print(f"  Project: {GOOGLE_CLOUD_PROJECT}")

## Usage in Analysis Notebooks

### Recommended: Use the helper module

**Make sure you've uploaded `aou_helpers.py` to the same directory as your notebooks!**

Then in any analysis notebook:

```python
import os
from aou_helpers import load_aou_env

# Load config and set os.environ
env = load_aou_env()

# Now use os.environ directly (standard pattern)
WORKSPACE_CDR = os.environ['WORKSPACE_CDR']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
WORKSPACE_TEMP_BUCKET = os.environ['WORKSPACE_TEMP_BUCKET']
GOOGLE_CLOUD_PROJECT = os.environ['GOOGLE_CLOUD_PROJECT']
```

### Alternative: Direct config file access

If you prefer to load the config file directly (without the helper):

```python
import json
import os

with open('/home/jupyter/workspace/.aou_config.json') as f:
    env = json.load(f)

# Update os.environ for compatibility
os.environ.update(env)

# Use os.environ
WORKSPACE_CDR = os.environ['WORKSPACE_CDR']
```

In [ ]:
# Example: Using the helper module
import os
from aou_helpers import load_aou_env

# Load config (this also sets os.environ)
env = load_aou_env()

dataset = os.environ['WORKSPACE_CDR']
bucket = os.environ['WORKSPACE_BUCKET']
temp_bucket = os.environ['WORKSPACE_TEMP_BUCKET']
gcproject = os.environ['GOOGLE_CLOUD_PROJECT']

# Verify variables are accessible via os.environ
print("Environment variables loaded successfully:")
print(f"  dataset: {dataset}")
print(f"  bucket: {bucket}")
print(f"  temp_bucket: {temp_bucket}")
print(f"\n✓ Ready to use in analysis notebooks!")